Q1  - AVG TEMPERATURE EVOLUTION OF:
 >> SPECIFIC COUNTRIES FULLFILING CERTAIN CONDITIONS > SHOW SUBSETTTING PART

 >> MAP, LINEPLOT, PERCENTUAL CHANGE
 
 >> CHALLENGE WHILE CREATING THE MAP

# 1. IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# 2. CREATING SQL CONNECTION ENGINE 

In [2]:
# SQL CONNECTION LIBRARIES
from dotenv import dotenv_values
from sqlalchemy import create_engine

In [3]:
# SQL LOGIN CREDENTDIALS ENV FILE 
config = dotenv_values(".env")
# SQL LOGIN VARIABLES PY
username = config['USER'] 
password = config['PASS']
host = config['HOST']
port = config['PORT']

In [4]:
# SQL CONNECTION LINK - TO CLIMATE DATABASE
url = f'postgresql://{username}:{password}@{host}:{port}/climate' 

In [5]:
# SQL CONNECTION ENGINE
engine = create_engine(url, echo=False) #ECHO false > avoiding SQL confirmation/output

# 3. RETRIEVING DATA FROM SQL DB 

In [63]:
with engine.begin() as conn: # Done with echo=True
    result = conn.execute(
    """SELECT 
        s.cn
        ,c.name
        ,c.alpha3	
        ,ya.staid
        ,s.staname
        ,s.coordinates
        ,ya.year
        ,ya.avg_year_temp
    FROM yearly_avg ya 
        JOIN stations s
        USING (staid)
        JOIN countries c
        ON c.alpha2=s.cn
    --WHERE
    --    s.cn IN ('SE','DE','FR')
    ;""")
    data = result.all()


The method `result.all()` reads all rows from the result object and returns a list
of tuples:

In [64]:
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332009 entries, 0 to 332008
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   cn             332009 non-null  object 
 1   name           332009 non-null  object 
 2   alpha3         332009 non-null  object 
 3   staid          332009 non-null  int64  
 4   staname        332009 non-null  object 
 5   coordinates    332009 non-null  object 
 6   year           332009 non-null  float64
 7   avg_year_temp  332009 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 20.3+ MB


In [65]:
df["year"]=df["year"].astype(int)
df["avg_year_temp"]=df["avg_year_temp"].astype(float)
df["avg_year_temp"]=df["avg_year_temp"]/10 

In [66]:
df['coordinates'].str.split(",",expand=True)
df[['lat', 'lon']]=pd.DataFrame(df['coordinates'].str.split(",", expand=True), index=df.index)

In [67]:
df["lat"]=df["lat"].str.replace("(","")
df["lon"]=df["lon"].str.replace(")","")
df["lat"]=df["lat"].astype(float)
df["lon"]=df["lon"].astype(float)

C:\Users\K_Erg\AppData\Local\Temp\ipykernel_13888\1294420609.py:1: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

C:\Users\K_Erg\AppData\Local\Temp\ipykernel_13888\1294420609.py:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [68]:
df

,cn,name,alpha3,staid,staname,coordinates,year,avg_year_temp,lat,lon
0,SE,Sweden,SWE,1,vaexjoe,"(56.86666666666667,14.8)",1860,0.000000,56.866667,14.800000
1,SE,Sweden,SWE,1,vaexjoe,"(56.86666666666667,14.8)",1862,0.000000,56.866667,14.800000
2,SE,Sweden,SWE,1,vaexjoe,"(56.86666666666667,14.8)",1864,0.000000,56.866667,14.800000
3,SE,Sweden,SWE,1,vaexjoe,"(56.86666666666667,14.8)",1865,0.000000,56.866667,14.800000
4,SE,Sweden,SWE,1,vaexjoe,"(56.86666666666667,14.8)",1866,0.000000,56.866667,14.800000
...,...,...,...,...,...,...,...,...,...,...
332004,PL,Poland,POL,25156,krosno,"(49.70666666666666,21.769166666666667)",2021,8.580548,49.706667,21.769167
332005,PL,Poland,POL,25156,krosno,"(49.70666666666666,21.769166666666667)",2022,10.816484,49.706667,21.769167
332006,NO,Norway,NOR,25157,ylja kraftverk,"(61.19694444444445,8.380555555555556)",2022,7.371495,61.196944,8.380556
332007,NO,Norway,NOR,25159,farkollen,"(69.80416666666666,28.993333333333332)",2022,10.138053,69.804167,28.993333


# 4. PLOTTING WITH MAPBOX 

## 4.1. DATA RESHAPING - PIVOT METHOD

In [70]:
df_piv_tab= df.pivot_table(values="avg_year_temp",index=["name","cn","alpha3","year"], aggfunc="mean")
df_piv_tab.head(300)

avg_year_temp
name    cn alpha3 year               
Albania AL ALB    1940      17.495273
                  1941      15.279726
                  1942      16.520000
                  1943      17.000822
                  1944      16.524262
...                               ...
Austria AT AUT    1924       6.021178
                  1925       6.342567
                  1926       8.075970
                  1927       7.572773
                  1928       7.719860

[300 rows x 1 columns]

In [71]:
df_piv_tab.reset_index(level=0, inplace=True)
df_piv_tab.reset_index(level=0, inplace=True)


In [72]:
df_piv_tab.reset_index(level=0, inplace=True)
df_piv_tab

,alpha3,cn,name,avg_year_temp
year,,,,
1940,ALB,AL,Albania,17.495273
1941,ALB,AL,Albania,15.279726
1942,ALB,AL,Albania,16.520000
1943,ALB,AL,Albania,17.000822
1944,ALB,AL,Albania,16.524262
...,...,...,...,...
2001,UZB,UZ,Uzbekistan,15.445944
2002,UZB,UZ,Uzbekistan,15.081288
2003,UZB,UZ,Uzbekistan,14.535114


In [73]:
df_piv_tab.reset_index(level=0, inplace=True)
df_piv_tab

,year,alpha3,cn,name,avg_year_temp
0,1940,ALB,AL,Albania,17.495273
1,1941,ALB,AL,Albania,15.279726
2,1942,ALB,AL,Albania,16.520000
3,1943,ALB,AL,Albania,17.000822
4,1944,ALB,AL,Albania,16.524262
...,...,...,...,...,...
5962,2001,UZB,UZ,Uzbekistan,15.445944
5963,2002,UZB,UZ,Uzbekistan,15.081288
5964,2003,UZB,UZ,Uzbekistan,14.535114
5965,2004,UZB,UZ,Uzbekistan,15.641940


## 4.2. EDA - TIME RANGE OF THE DATASET/PER COUNTRY

In [ ]:
agg_dict = { 'year' : ["min","max"]}
test= df_piv_tab.groupby(["name"]).agg(agg_dict)
test

In [105]:
test.reset_index(col_level=1, inplace=True)

In [ ]:
test.columns=["country","min_year","max_year"]
test

In [154]:
test["diff"] =  test["max_year"] - test['min_year']
test

,country,min_year,max_year,diff
0,Albania,1940,2000,60
1,Algeria,1940,1998,58
2,Armenia,1894,2005,111
3,Austria,1855,2022,167
4,Azerbaijan,1882,1991,109
5,Belarus,1881,2022,141
6,Belgium,2020,2022,2
7,Bosnia and Herzegovina,1901,2022,121
8,Croatia,1858,2022,164
9,Cyprus,1972,2022,50


## 4.3. DATASET SUBSETTING

In [165]:
test_subs = test[
    (test["diff"]>=100) # MEASUREMENT OF AT LEAST 100 YEARS
    & (test["max_year"]==2022) # UNTIL 2022
    & (test["min_year"]<=1922) # AT LEAST FROM 1922 ONWARRDS
    ]

print(len(test_subs)) # HOW MANY COUNTRIES FULLFILL THAT?

my_countries=test_subs["country"].tolist() # LIST OF THE COUNTRIES

my_countries

28


['Austria',
 'Belarus',
 'Bosnia and Herzegovina',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greenland',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'Slovenia',
 'Spain',
 'Svalbard and Jan Mayen',
 'Sweden',
 'Switzerland',
 'Ukraine',
 'United Kingdom']

In [87]:
# df_piv_tab.drop(["index"], axis = 1, inplace = True)
df_piv_tab.sort_values(by=["year"])
df_piv_tab.sort_values(by=["alpha3"])


,year,alpha3,cn,name,avg_year_temp
0,1940,ALB,AL,Albania,17.495273
32,1973,ALB,AL,Albania,15.193014
33,1974,ALB,AL,Albania,15.168630
34,1975,ALB,AL,Albania,15.267808
35,1976,ALB,AL,Albania,14.617486
...,...,...,...,...,...
5885,1924,UZB,UZ,Uzbekistan,13.336202
5884,1923,UZB,UZ,Uzbekistan,12.982055
5883,1922,UZB,UZ,Uzbekistan,14.242279
5909,1948,UZB,UZ,Uzbekistan,13.407149


In [173]:
df_piv_tab_sub = df_piv_tab[
  (df_piv_tab["name"].isin(
   ['Austria',
 'Belarus',
 'Bosnia and Herzegovina',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greenland',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Ukraine',
 'United Kingdom']))
 & (df_piv_tab["year"]>=1922)
]

In [174]:
df_piv_tab_sub

,year,alpha3,cn,name,avg_year_temp
293,1922,AUT,AT,Austria,6.153465
294,1923,AUT,AT,Austria,7.578594
295,1924,AUT,AT,Austria,6.021178
296,1925,AUT,AT,Austria,6.342567
297,1926,AUT,AT,Austria,8.075970
...,...,...,...,...,...
5850,2018,GBR,GB,United Kingdom,10.299193
5851,2019,GBR,GB,United Kingdom,10.089332
5852,2020,GBR,GB,United Kingdom,10.248778
5853,2021,GBR,GB,United Kingdom,9.681968


# 5. DATA PLOTTING

## 5.1. CHOROPLETH MAP

In [205]:
fig = px.choropleth(
    data_frame=df_piv_tab_sub, 
    locations="alpha3", # cn > iso alpha code
    projection='orthographic',
    color="avg_year_temp", 
    hover_name="name", # column to add to hover information
    color_continuous_scale=px.colors.sequential.Viridis,
    animation_frame = 'year', # iteration over the years
    animation_group = 'name', # country name, where should the change happen
    template = 'plotly'
    ,height = 700
    ,width = 1000
    ,range_color=(min(df_piv_tab_sub["avg_year_temp"]), max(df_piv_tab_sub["avg_year_temp"]))
    ,title ="MEAN TEMPERATURE EVOLUTION")
fig.show()

fig.write_html('avg_year_temp_map.html')

## 5.2. LINE PLOT OF AVG TEMPERATURE EVOLUTION - SUBJECT-MATTER EXPERT HELP NEEDED?

In [200]:

fig = px.line(
    df_piv_tab_sub
    , x="year"
    , y="avg_year_temp"
    , color='name'
    , title= "MEAN TEMPERATURE EVOLUTION")

fig.show()


fig.write_html('avg_year_temp_line_plot.html')

## 5.3. PERCENTUAL CHANGE  - PURE NUMBERS ANALYSIS 

In [206]:
df_piv_tab_sub_reshape = df_piv_tab_sub.pivot(index="year",columns="name",values="avg_year_temp")
df_piv_tab_sub_reshape

name,Austria,Belarus,Bosnia and Herzegovina,Croatia,Czech Republic,Denmark,Estonia,Finland,France,Germany,...,Poland,Portugal,Romania,Russia,Slovenia,Spain,Sweden,Switzerland,Ukraine,United Kingdom
year,,,,,,,,,,,,,,,,,,,,,
1922,6.153465,7.044505,9.084384,9.896621,8.382740,6.960274,3.944072,-0.516301,11.947959,7.121677,...,NaN,15.739123,9.394608,0.894978,9.855068,14.849911,2.746872,7.619434,9.287864,8.510523
1923,7.578594,7.484505,10.782153,10.918174,9.317260,7.275068,4.484522,-0.265753,12.121076,7.981092,...,NaN,16.313663,12.062756,0.937349,10.802466,14.916535,2.910403,8.386117,10.266398,8.791389
1924,6.021178,5.375410,8.966667,9.793534,8.632514,7.338547,5.164208,0.468187,11.280868,7.535283,...,NaN,15.596822,10.387650,0.605916,9.937705,15.311375,4.236025,7.793320,9.316253,8.689613
1925,6.342567,7.312877,9.426301,10.502648,9.550959,8.040000,5.883616,0.276438,11.892521,8.372480,...,NaN,15.579054,11.207114,1.498437,9.915569,14.438877,4.349726,7.838289,10.326895,8.997338
1926,8.075970,6.229452,10.129006,13.165922,9.971233,7.813973,4.472723,-0.821063,12.673371,8.653131,...,NaN,16.847128,11.513824,0.474987,10.798904,15.805488,3.836917,8.876368,8.715065,9.367048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,9.950397,9.393151,11.496438,14.272329,12.873425,9.791368,7.209279,4.480426,13.648513,10.489720,...,9.076591,17.076374,10.754235,0.283814,10.511046,14.431960,5.514055,9.606575,9.500865,10.299193
2019,9.775367,10.118356,11.711507,14.247187,12.683832,9.849122,7.586594,4.105483,13.436223,10.225119,...,9.309996,17.451099,11.336105,-4.711087,10.614147,15.348863,5.416043,9.174231,10.297331,10.089332
2020,9.382191,9.922131,11.435519,13.863072,12.307923,10.239593,8.427931,5.664792,13.702721,10.457709,...,9.106752,19.127170,11.340817,4.423480,10.223792,14.936410,6.745546,9.399727,10.309125,10.248778
